In [1]:
#!pip install ultralytics

In [2]:
#!pip install ultralytics opencv-python torch torchvision

In [3]:
from ultralytics import YOLO
import cv2
import os


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\GURU\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [29]:
import numpy as np
# Function to correct image rotation
def auto_rotate(image):
    try:
        # Get metadata (some videos store rotation info)
        exif = image._getexif()
        if exif is not None:
            orientation = exif.get(0x0112)  # EXIF tag for orientation
            if orientation == 3:
                image = cv2.rotate(image, cv2.ROTATE_180)
            elif orientation == 6:
                image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
            elif orientation == 8:
                image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    except AttributeError:
        pass  # No EXIF data, continue normally
    return image

In [34]:
def fix_rotation(image):
    h, w = image.shape[:2]
    if h > w:  # Likely rotated, so fix it
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)  # Rotate to landscape
    return image

In [35]:
# Load the YOLOv8 model
model = YOLO("best.pt")  # Ensure best.pt is in the current directory

# Function to classify a frame

# Function to classify a frame
def classify_frame(frame):
    frame = fix_rotation(frame)  # Fix rotation
    frame_resized = cv2.resize(frame, (640, 640))  # Resize to YOLO expected size
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)  # Convert for YOLO

    results = model(frame_rgb)
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])  # Get predicted class
            confidence = float(box.conf[0])  # Get confidence score
            if confidence > 0.5:
                return class_id, confidence  # Return detected class
    return None, None  # No detection

In [36]:
# Process video and save drowsy and non-drowsy frames
def process_video(video_path, drowsy_folder, non_drowsy_folder, frame_interval=10):
    os.makedirs(drowsy_folder, exist_ok=True)
    os.makedirs(non_drowsy_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    drowsy_count = 0
    non_drowsy_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = fix_rotation(frame)  # Fix rotation before processing

        if frame_count % frame_interval == 0:
            class_id, confidence = classify_frame(frame)

            if class_id == 1 and confidence > 0.5:
                frame_path = os.path.join(drowsy_folder, f"drowsy_{drowsy_count}.jpg")
                drowsy_count += 1
            else:
                frame_path = os.path.join(non_drowsy_folder, f"non_drowsy_{non_drowsy_count}.jpg")
                non_drowsy_count += 1

            cv2.imwrite(frame_path, cv2.resize(frame, (640, 640)))  # Save properly rotated frame

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [8]:
from ultralytics import YOLO

model = YOLO("best.pt")
print("Model loaded successfully!")


Model loaded successfully!


In [9]:
print(model.names)  # Check the class labels in the model


{0: 'awake', 1: 'drowsy'}


In [13]:
print(model.model.args['imgsz'])  # Correct way to get input size


640


In [51]:
from ultralytics import YOLO
import cv2

# Load the model
model = YOLO("best.pt")

# Test on a single frame
img_path = cv2.imread(r"D:\Major Project\Rasp\Data\test\drowsy\Drowsy (2202).jpg")  # Replace with a sample image from your dataset
image_resized = cv2.resize(img_path, (640, 640))
results = model(img_path, show=True)  # Show the image with detections
print(results)
cv2.waitKey(0)  # Wait for a key press
cv2.destroyAllWindows()



0: 384x640 1 awake, 137.4ms
Speed: 3.5ms preprocess, 137.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'awake', 1: 'drowsy'}
obb: None
orig_img: array([[[120, 138, 145],
        [121, 140, 147],
        [117, 136, 144],
        ...,
        [103, 127, 125],
        [101, 129, 123],
        [ 99, 131, 120]],

       [[119, 139, 144],
        [122, 144, 149],
        [117, 142, 146],
        ...,
        [101, 127, 127],
        [ 98, 129, 122],
        [ 95, 129, 118]],

       [[116, 145, 142],
        [115, 146, 143],
        [113, 144, 143],
        ...,
        [ 97, 129, 128],
        [ 94, 131, 123],
        [ 91, 131, 120]],

       ...,

       [[ 19,  76,  73],
        [ 18,  77,  73],
        [ 18,  75,  72],
        ...,
        [212, 243, 210],
        [213, 241, 211],
        [213, 242, 209]],

       [[ 1

In [38]:
video = r"D:\Major Project\Final Proper\Video_Data\Fold1_part1\02\0.mov"
# Example usage
process_video(video, "drowsy_frames", "non_drowsy_frames", frame_interval=10)


0: 640x640 (no detections), 255.9ms
Speed: 4.2ms preprocess, 255.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 188.1ms
Speed: 3.2ms preprocess, 188.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 197.1ms
Speed: 3.7ms preprocess, 197.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 220.0ms
Speed: 4.5ms preprocess, 220.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 211.8ms
Speed: 4.0ms preprocess, 211.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 193.7ms
Speed: 4.2ms preprocess, 193.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 232.2ms
Speed: 3.7ms preprocess, 232.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 171.7ms
Speed: 2.8ms prepr

KeyboardInterrupt: 